### Exploring the Kalman Gain matrix
In this exercise, you start breaking down how individual and sets of observations impact your flux estimates. For this exercise, the NOAA CarbonTracker website, https://gml.noaa.gov/ccgg/carbontracker/co2tser.php will be helpful to map observation ids for in situ data to geographic locations in the world.

- Do you see large difference in which sites/observations are more influential in the northern midlatititudes, e.g. Europe and North America, vs the tropics, e.g. Africa and S. America?  Any ideas why?
  
- For state elements controlling the flux of Northern high latitude regions, e.g. Boreal North America, are satellite or in situ data more valuable? or does it depend?

- What about places like Europe or Temperate North America?

- Why do places in the southern hemisphere, at same latitude away from equator as U.S. and Europe, e.g. Temp South America, look different than their northern counterparts would you guess?


#### Setting up Environment for Computing¶
This cell simply looks for whether we are on GHGHub (or local) and sets up environment, including directory references and libraries.

In [ ]:
######################################################################
#-- Read settings for location of data and set up, NO NEED TO CHANGE
######################################################################
orig_dir = getwd()
require(yaml,warn.conflicts = FALSE)
dat = yaml.load_file("../site_settings.yml")
Rcode_dir <- getwd()
data_dir = paste(dat$global_paths$input_folder,"/",sep="")
output_dir = paste(dat$global_paths$output_folder,"/",sep="")

print(paste("Using",data_dir,"for data directory"))
print(paste("Using",output_dir,"for output directory"))

#--  Load utility code file w/ setup()
source(file.path(Rcode_dir,"util_code_032024.R"))
setup()

In [ ]:
###############################################
#--  Load sensitivity matrices 
###############################################

load(file.path(data_dir,"jacobians/","trunc_full_jacob_030624_with_dimnames_sib4_4x5_mask.rda"))
load(file.path(data_dir,"jacobians/","jacob_bgd_060524.rda"))

#-- Difference in forward runs from GEOS-CHem resulted in CO2 vs C diff in mass is why 12/44 is here (note)
#-- Assign the jacob objects to H to match notation
H <- jacob * 12/44
H_bgd <- jacob_bgd 
rm(jacob);rm(jacob_bgd)

#-- These represent the fossil and biomass burning contributions to the observations (from fixed emission runs)
fire_fixed <- H_bgd[,2]
fossil_fixed <- H_bgd[,3]
###################################################################
#-- END END END ***Parent Directory and code for ALL inversions***
###################################################################

In [ ]:

##################################################################
#- Inversion #1   *************************
##################################################################

#################################
#- Target truth in state space
#################################

##################################################################
#-- This array holds ratios of OCO2v10MIP fluxes and SiB4 fluxes
#-- as examples of "scalings" to be recovered. It also holds corresponding
#-- differences if the inversion attempts to directly solve for flux
#-- truth_array(24 months, 23 transcom, 98 inversions, (ratio, difference) )
##################################################################

#-- Don't Change
#load("/projects/sandbox/inversion_workshop_scripts/truth_array.rda")
load(file.path(data_dir,"misc/truth_array.rda"))
#-- pulling out NA transcom region and subset to scalar vs flux adj
truth_array = truth_array[,-1,,1]
#-- Don't Change


#--  Choose our state from inversion list, option #1, and "truncate" to -1 and 1
inversion_number =1   #  choose this between 1 and 98
state_vector_true= tm(as.vector(- truth_array[,,inversion_number]),-1,1)

#-- Alternatively choose a "different" true state like the below ones
#-- The first just means the truth IS the prior, the second has a simple structure
#-- Land regions fluxes are (1+0.5) * prior guess and ocean fluxes are (1- 0.5) * prior guess.
#state_vector_true = c(rep(0,24*11),rep(0,24*11))
#state_vector_true = c(rep(0.5,24*11),rep(-0.5,24*11))

In [ ]:
#########################################################
# Generate a prior flux covariance matrix Sx
# These first two lines form "diagonal" of Sx, e.g. marginal variances
# Long term, a catalog of predefined choices is best here I think
#########################################################
land_prior_sd = 0.5   #-- free to set this, implies you think "truth" for land is within +/- 3*this
ocean_prior_sd = 1    #-- free to set this, implies you think "truth" for ocean is within +/- 3*this

##############################################################################
#-- This is the structure of the 24 month subblock for each land/ocean region
#-- induce temporal correlations
##############################################################################

#-- This will set up a prior temporal correlation, 
#-- free to set month_to_month_correlation between 0 (independent) and 1
month_to_month_correlation = 0.5
sigma = bdiag(rep(list(ar_covariance(24, month_to_month_correlation)), 22))  #-- free to set 


#################################################
#-- scale by variance for land/ocean (set diagonal of matrix)
#-- This simply puts together pieces above
#################################################
var_scaling_diagonal = diag(c(rep(land_prior_sd,24*11),rep(ocean_prior_sd,24*11)))

Sx = as.matrix(var_scaling_diagonal %*% sigma %*% t(var_scaling_diagonal))

#-- This is an alternative state_vector_true based *exactly* upon the prior covariance matrix
#-- as opposed to being able to pick your "truth" separately from your assumed dist where "truth" lives
#-- Probably don't want to change this unless you know what you are doing
#state_vector_true = t(rmvnorm(n=1,mean=rep(0,528),sigma=sigma))


#### Choose which observations you want to assimilate
Or in other words, which observations will be used to optimize/estimate the unknown fluxes.  This problem is somewhat over determined with over a million observations to constrain a 528 element state.  With that in mind, small observation errors and LOTS of observations used should "nail the unknown" solution quite well. The goal here is to create a vector of TRUE/FALSE of length equal to the total number of observations described in the sensitivity matrix we loaded above ( 1156383 ). The obs_catalog is a data.frame (think matrix of 'items'), with information about each observation and can be used to build a subset.

In [ ]:
####################################################################################
#-- WHICH obs do you want to use in the inversion? 
#-- examples of selecting on stations, type of data, lat/lon box,etc
####################################################################################

#load(file.path(data_dir,"obs/obs_catalog_030624.rda")) # obs_catalog object
load(file.path(data_dir,"obs/obs_catalog_042424_unit_pulse_hour_timestamp_witherrors_withdates.rda")) 


subset_indicator_obs=rep(FALSE,dim(H)[1])
#subset_indicator_obs=rep(TRUE,dim(jacob)[1])

#subset_indicator_obs=c(rep(TRUE,156383),rep(FALSE,1000000))

############################
#-- SAMPLE BY TYPE EXAMPLE
############################
#subset_indicator_obs[obs_catalog$TYPE == "TCCON"] = TRUE
#subset_indicator_obs1 = rep(FALSE,length(subset_indicator_obs))
#subset_indicator_obs2 = rep(FALSE,length(subset_indicator_obs))

#subset_indicator_obs1[obs_catalog$TYPE == "OCO2"] = TRUE
#subset_indicator_obs2[seq(1,1156383,by=2)] = TRUE
#subset_indicator_obs = subset_indicator_obs1 & subset_indicator_obs2


############################
#-- SAMPLE BY NOAA STATION EXAMPLE
############################
#subset_indicator_obs[grep("spo",obs_catalog$ID)] = TRUE
#subset_indicator_obs[grep("lef",obs_catalog$ID)] = TRUE

############################
#-- SAMPLE BY TIME EXAMPLE
############################
#subset_indicator_obs[obs_catalog$TIME > 8738000] = TRUE

############################
#-- SAMPLE BY LON & LAT EXAMPLE
############################
#subset_indicator_obs[obs_catalog$LON < -10 & obs_catalog$LAT > 10] = TRUE

#subset_indicator_obs=c(rep(TRUE,1156382),rep(FALSE,1))
subset_indicator_obs[seq(1,1156383,by=5)] = TRUE
#table(subset_indicator_obs)



############################
#-- Downsample if necessary
############################

if(sum(subset_indicator_obs) > 0.5*length(subset_indicator_obs)) {
  new_ind = rep(FALSE,length(subset_indicator_obs))
  new_ind[sample(x=grep(TRUE,subset_indicator_obs),size=floor(0.5*length(subset_indicator_obs)))] = TRUE
  print(paste("downsampling from",sum(subset_indicator_obs),"to",
              floor(0.5*length(subset_indicator_obs)),"observations"))
  subset_indicator_obs = new_ind
    }

#-- LEAVE THIS AS IT SUMMARIZES THE NUMBER OF OBS USED
print(paste("using",sum(subset_indicator_obs),"of",length(subset_indicator_obs),"observations"))

In [ ]:
##########################################################
#-- sd for Gaussian i.i.d. errors, jacob is sens matrix
##########################################################
#R_diagonal_in = rep(3,(dim(jacob)[1]))
Sz_diagonal_in = obs_catalog$SD

In [ ]:
#############################################################
#-- Generate obs, 'y',  set.seed() ????
#-- currently leaving out bgd and all fixed
#-- non-optimizable contributions including fire and fossil
#############################################################

err = rnorm(length(Sz_diagonal_in),sd=Sz_diagonal_in)
z_in = H %*% (1+state_vector_true) + err

z_in_centered = H %*% (state_vector_true) + err

$$
\newcommand{\transpose}[1]{{#1^{\scriptscriptstyle T}}} 
J(x) = \transpose{(x_0 - x)} {\Sigma_x
}^{-1}(x_0 - x) + \transpose{(z - Hx)} {\Sigma_z}^{-1}(z - Hx)\\
$$

$$
\newcommand{\transpose}[1]{{#1^{\scriptscriptstyle T}}} 
\hat{x} = (\transpose{H}{\Sigma_z}^{-1}H + {\Sigma_x}^{-1})^{-1}(\transpose{H}{\Sigma_z}^{-1}(z-Hx)+{\Sigma_x}^{-1}x_0)
$$

$$
\newcommand{\transpose}[1]{{#1^{\scriptscriptstyle T}}} 
\Sigma_{\hat{x}} = {({\Sigma_x}^{-1} + \transpose{H}{\Sigma_z}^{-1}H )}^{-1}
$$







In [ ]:
############################
#-- Run the actual inversion
############################

ret2  = invert_clean_notation(H=H,Sz_diagonal=Sz_diagonal_in,Sx=Sx,z=z_in,H_bgd=H_bgd,
                    subset_indicator_obs=subset_indicator_obs,DOF=FALSE,output_Kalman_Gain=TRUE,
                     state_vector_true=state_vector_true)



#### Looking at importance via Kalman Gain (find the biggest value in Gain matrix)
Here we are identifying the LARGEST value of Kalman Gain, across all observations for each "state" element. This most likely is an observation which is in the geographic vicinity of the state element or downwind.

In [ ]:
#-- This code looks through Kalman Gain for observation with largest gain in the month of interest for that region
max_indices = apply(ret2$diags$KGAIN,1,FUN=function(x){(which(abs(x)==max(abs(x))))[1]})
max_indices = as.numeric(as.vector(unlist(max_indices)))
sts = obs_catalog$ID[subset_indicator_obs][max_indices]
typ = obs_catalog$TYPE[subset_indicator_obs][max_indices]

for(i in 1:22){
    for(j in 1:24){
    print(paste(gsub(" ","",transcom_names[i])," month:",j," TYPE:",typ[(i-1)*24+j],"  ",sts[(i-1)*24+j],sep=""))    
    }
}

#### Looking at importance via Kalman Gain (find which type of obs has biggest *average* impact on state)
Here we are plotting the average of the K*(model-z), the "state adjustment", across different observation types. For a state element, e.g. North America Temperate for Jul 2016, we average over the state adjustment by each observation type. This gives average impact of the type of observation towards the state. *Values are normalized to sum to one across each month*. Note that this is performed for each state element across ALL global observations available.

In [ ]:
state_adjustment_per_obs = t(apply(ret2$diags$KGAIN,1,FUN=function(x){x* z_in_centered[subset_indicator_obs]}))

options(repr.plot.width=12, repr.plot.height=8)

for(i in 1:22){
 out = apply(abs(state_adjustment_per_obs[((i-1)*24+1):((i)*24),]),1,
             FUN=function(x){aggregate(x,list(obs_catalog$TYPE[subset_indicator_obs]),mean)})
 out2 = cbind(IS = sapply(out,FUN=function(x){x$x[1]}),
              OCO2 = sapply(out,FUN=function(x){x$x[2]}),
             TCCON = sapply(out,FUN=function(x){x$x[3]}))
    out3 = as.data.frame(t(apply(out2,1,FUN=function(x){return(x/sum(x))})))
    plot(1:24,out3$IS,col="blue",type="b",ylim=c(0,1),xaxt="none",ylab="Relative impact on state adjustment",xlab="Date",
         main=paste("Average impact of 1 observation on state adjustment for:",transcom_names[i]))
    axis(side = 1,labels = dts[seq(1,24,by=6)],at=(1:24)[seq(1,24,by=6)],cex=0.9)
    points(1:24,out3$OCO2,col="red",type="b",cex=1.1)
    points(1:24,out3$TCCON,col="orange",type="b")
    legend(1,1,c("IS","OCO2","TCCON"),col=c("blue","red","orange"),lty=1,pch=1,bg="white")
}

#### Looking at importance via Kalman Gain(find which type of obs has biggest *total* impact on state)
Similarly, we are not plotting the *sum* of the K*(model-z), the "state adjustment", across different observation types. For a state element, e.g. North America Temperate for Jul 2016, this gives the total impact of the type of observation towards the state, i.e. incorporating the number of observations per type.  *Values are normalized to sum to one across each month*.  Note that this is performed for each state element across ALL global observations available.

In [ ]:
state_adjustment_per_obs = t(apply(ret2$diags$KGAIN,1,FUN=function(x){x* z_in_centered[subset_indicator_obs]}))
options(repr.plot.width=12, repr.plot.height=8)

for(i in 1:22){
 out = apply(abs(state_adjustment_per_obs[((i-1)*24+1):((i)*24),]),1,FUN=function(x){aggregate(x,list(obs_catalog$TYPE[subset_indicator_obs]),sum)})
 out2 = cbind(IS = sapply(out,FUN=function(x){x$x[1]}),
              OCO2 = sapply(out,FUN=function(x){x$x[2]}),
             TCCON = sapply(out,FUN=function(x){x$x[3]}))
    out3 = as.data.frame(t(apply(out2,1,FUN=function(x){return(x/sum(x))})))
    plot(1:24,out3$IS,col="blue",type="b",ylim=c(0,1),xaxt="none",ylab="Relative impact on state adjustment",xlab="Date",
         main=transcom_names[i])
    axis(side = 1,labels = dts[seq(1,24,by=6)],at=(1:24)[seq(1,24,by=6)],cex=0.9)
    points(1:24,out3$OCO2,col="red",type="b",cex=1.1)
    points(1:24,out3$TCCON,col="orange",type="b")
    legend(1,1,c("IS","OCO2","TCCON"),col=c("blue","red","orange"),lty=1,pch=1,bg="white")
}

In [ ]:
sessionInfo()